In [2]:
!nodetool status

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address     Load       Tokens  Owns (effective)  Host ID                               Rack 
UN  172.21.0.4  70.25 KiB  16      66.2%             739395b2-9b2a-4828-a1a2-a85a7e358b24  rack1
UN  172.21.0.2  70.25 KiB  16      62.4%             d9d860d3-d1f1-4282-85f4-efd30b5b1258  rack1
UN  172.21.0.3  70.25 KiB  16      71.4%             b6057742-477a-4263-8869-1dc8926cb85e  rack1



In [13]:
#Connect to the Cassandra cluster
from cassandra.cluster import Cluster
cluster = Cluster(['p6-db-1', 'p6-db-2', 'p6-db-3'])
cass = cluster.connect()

In [14]:
#q1
#drop a weather keyspace if it already exists
cass.execute("drop keyspace if exists weather;")
#create a weather keyspace with 3x replication
cass.execute("create keyspace weather with replication={'class': 'SimpleStrategy', 'replication_factor': 3};")
#inside weather, create a station_record type containing two ints: tmin and tmax
cass.execute("""
CREATE TYPE
weather.station_record (
    tmin int,
    tmax int
)
""")
#inside weather, create a stations table
cass.execute("""
CREATE TABLE weather.stations (
    id text,
    name TEXT STATIC,
    date date,
    record station_record,
    state TEXT,
    PRIMARY KEY (id,date)
) WITH CLUSTERING ORDER BY (date ASC)
""")

print(cass.execute("describe table weather.stations ").one().create_statement)

In [ ]:
#Create a local Spark session
from pyspark.sql import SparkSession
spark = (SparkSession.builder
         .appName("p6")
         .config('spark.jars.packages', 'com.datastax.spark:spark-cassandra-connector_2.12:3.4.0')
         .config("spark.sql.extensions", "com.datastax.spark.connector.CassandraSparkExtensions")
         .getOrCreate())

In [ ]:
#q2